# Machine Learning - BERT

**Relator: Felipe Mesa Abraham** 

Correo: femesa@udec.cl

En la siguiente actividad, se trabajará con modelos de tipo transformer, siguiendo una arquitectura BERT. Puede basarse en la documentación del paquete  [Transformers](https://huggingface.co/transformers/) y  consultar las diferentes opciones de modelos pre entrenados en la pagina de [Hugging Face](https://huggingface.co/) 

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 17.0 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 596 kB 36.3 MB/s 
     |████████████████████████████████| 59 kB 4.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install datasets

     |████████████████████████████████| 290 kB 3.2 MB/s 
     |████████████████████████████████| 132 kB 43.0 MB/s 
     |████████████████████████████████| 243 kB 46.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.5 MB/s 
     |████████████████████████████████| 271 kB 39.8 MB/s 
     |████████████████████████████████| 160 kB 39.9 MB/s 
     |████████████████████████████████| 192 kB 42.7 MB/s 


## Ejemplo 1: Cargando un modelo pre entrenado

In [3]:
from transformers import pipeline

In [4]:
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
texto = 'Compre online 01 cama, la que debía llegar hoy, lo cual hasta el momento no eh obtenido respuesta alguna, envié un correo respecto a saber cuando llegaría, no obstante aquello cero respuesta al respecto, me parece pésimo el servicio que prestan, debiesen contratar personas que realmente den una respuesta acorde como cliente, y uno se pudiese sentir a gusto, con las compras que realiza, no estar preocupado que probablemente mi pedido no llegara, siendo que ya esta pagado, me parece súper irresponsables y juegan con eso, con el tiempo de la gente...y no son ni siquiera capaces de dar una respuesta acorde.'
classifier(texto)

[{'label': '1 star', 'score': 0.7212204337120056}]

In [6]:
# Importamos librerias
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow import keras
from datasets import load_dataset

In [7]:
from transformers import AutoTokenizer

In [8]:
from transformers import TFAutoModelForSequenceClassification



---



## Ejemplo 2: Haciendo "Fine Tuning" usando un set de datos

In [6]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [10]:
tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

In [11]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [13]:
tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

In [14]:
train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

Epoch 1/3
125/125 [==============================] - 318s 2s/step - loss: 0.7424 - sparse_categorical_accuracy: 0.5200 - val_loss: 0.6929 - val_sparse_categorical_accuracy: 0.5120
Epoch 2/3
125/125 [==============================] - 292s 2s/step - loss: 0.7445 - sparse_categorical_accuracy: 0.5200 - val_loss: 0.8269 - val_sparse_categorical_accuracy: 0.5120
Epoch 3/3
125/125 [==============================] - 291s 2s/step - loss: 0.7335 - sparse_categorical_accuracy: 0.5130 - val_loss: 0.9301 - val_sparse_categorical_accuracy: 0.5120
